<a href="https://colab.research.google.com/github/bmurphy2019/groupProjectAi/blob/main/AlzheimersAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint



data_cross = pd.read_csv("/content/oasis_cross-sectional.csv")
data_long = pd.read_csv("/content/oasis_longitudinal.csv")

data_cross.columns = ['ID','Gender','Hand','Age','Educ','SES','MMSE','CDR','eTIV','nWBV','ASF','Delay']

data_long.columns = ['Subject ID','MRI ID','Group','Visit','MR Delay','Gender','Hand','Age','EDUC',
                      'SES','MMSE','CDR','eTIV','nWBV','ASF']

data_long.isna().sum()
data_cross.dropna(subset=['CDR'],inplace=True)

data_cross.drop(columns=['ID','Delay'],inplace=True)
data_long = data_long.rename(columns={'EDUC':'Educ'})
data_long.drop(columns=['Subject ID','MRI ID','Group','Visit','MR Delay'],inplace=True)

data = pd.concat([data_cross,data_long])


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['CDR'] = le.fit_transform(data['CDR'].values)

data = data.drop(data[data['CDR']==3].index)

data['Gender'] = data.Gender.map({'M': 0, 'F': 1})
data['Hand'] = data.Hand.map({'R': 0, 'L': 1})
y = data.pop('CDR')
X = data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

model = Sequential()
print(type(X-X_train))
model.add(Dense(9, activation = 'relu', input_shape = X_train.iloc[0].shape))

model.add(Dense(10, activation='relu'))

model.add(Dense(4, activation='relu')) 

model.add(Dense(1, activation='sigmoid')) 
model.summary()


model.compile(optimizer= 'adam' ,loss='binary_crossentropy',metrics=['acc'])

checkpointer = ModelCheckpoint('Alzheimers.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
history=model.fit(X_train, y_train, batch_size=16, epochs=350, validation_data=(X_test, y_test), callbacks=[checkpointer])

<class 'pandas.core.frame.DataFrame'>
Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 9)                 90        
                                                                 
 dense_91 (Dense)            (None, 10)                100       
                                                                 
 dense_92 (Dense)            (None, 4)                 44        
                                                                 
 dense_93 (Dense)            (None, 1)                 5         
                                                                 
Total params: 239
Trainable params: 239
Non-trainable params: 0
_________________________________________________________________
Epoch 1/350
20/30 [===================>..........] - ETA: 0s - loss: nan - acc: 0.5500     
Epoch 00001: val_acc improved from -inf to 0.56667, savin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
present_model = keras.models.load_model('Alzheimers.h5')
print("Accuracy of our model on test data : " , present_model.evaluate(X_test,y_test)[1]*100 , "%")


4/4 [==============================] - 0s 3ms/step - loss: nan - acc: 0.5667
Accuracy of our model on test data :  56.66666626930237 %


In [60]:
print("X train",X_train)
print("Y train",y_train)
print("X test",X_test)
print("X test",y_test)

X train      Gender  Hand  Age  Educ  SES  MMSE  eTIV   nWBV    ASF
151       0     0   84  17.0  1.0  29.0  1707  0.717  1.028
308       1     0   75  12.0  3.0  27.0  1357  0.714  1.293
346       0     0   84  16.0  2.0  30.0  1404  0.710  1.250
265       0     0   86  18.0  1.0  30.0  1670  0.669  1.051
107       1     0   92  23.0  1.0  29.0  1454  0.701  1.207
..      ...   ...  ...   ...  ...   ...   ...    ...    ...
46        0     0   84  12.0  3.0  27.0  1445  0.695  1.214
17        1     0   89   5.0  1.0  30.0  1536  0.715  1.142
128       0     0   68  18.0  1.0  30.0  1654  0.747  1.061
342       1     0   58   2.0  2.0  30.0  1418  0.821  1.238
333       1     0   71  18.0  1.0  30.0  1426  0.731  1.231

[479 rows x 9 columns]
Y train 151    0
308    2
346    0
265    1
107    0
      ..
46     1
17     0
128    0
342    0
333    0
Name: CDR, Length: 479, dtype: int64
X test      Gender  Hand  Age  Educ  SES  MMSE  eTIV   nWBV    ASF
232       1     0   78   3.0  2.0  29